In [2]:
from dotenv import load_dotenv, find_dotenv
import os
import pprint
load_dotenv(find_dotenv())

from rich import print

# Character Text Splitting Algorithm

In [3]:
from langchain_core.documents import Document
text = """
The astronauts on the International Space Station conducted a spacewalk to repair a malfunctioning solar panel. 
The aroma of freshly baked croissants wafted through the charming French bakery. 
The new policy aimed to reduce carbon emissions by 50% within the next decade. 
The ancient Egyptian pharaohs were known for their elaborate headdresses and ornate jewelry."""

# Manual Splitting
chunks = []
chunk_size = 35 # Characters
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]
print(documents)

[
    Document(metadata={'source': 'local'}, page_content='\nThe astronauts on the Internationa'),
    Document(metadata={'source': 'local'}, page_content='l Space Station conducted a spacewa'),
    Document(metadata={'source': 'local'}, page_content='lk to repair a malfunctioning solar'),
    Document(metadata={'source': 'local'}, page_content=' panel. \nThe aroma of freshly baked'),
    Document(metadata={'source': 'local'}, page_content=' croissants wafted through the char'),
    Document(metadata={'source': 'local'}, page_content='ming French bakery. \nThe new policy'),
    Document(metadata={'source': 'local'}, page_content=' aimed to reduce carbon emissions b'),
    Document(metadata={'source': 'local'}, page_content='y 50% within the next decade. \nThe '),
    Document(metadata={'source': 'local'}, page_content='ancient Egyptian pharaohs were know'),
    Document(metadata={'source': 'local'}, page_content='n for their elaborate headdresses a'),
    Document(metadata={'source': 'local'}, page_content='nd ornate jewelry.')
]

# Character Text Splitting Library

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', strip_whitespace=False)
documents = text_splitter.create_documents([text])
print(documents)

[
    Document(metadata={}, page_content='\nThe astronauts on the Internationa'),
    Document(metadata={}, page_content='l Space Station conducted a spacewa'),
    Document(metadata={}, page_content='lk to repair a malfunctioning solar'),
    Document(metadata={}, page_content=' panel. \nThe aroma of freshly baked'),
    Document(metadata={}, page_content=' croissants wafted through the char'),
    Document(metadata={}, page_content='ming French bakery. \nThe new policy'),
    Document(metadata={}, page_content=' aimed to reduce carbon emissions b'),
    Document(metadata={}, page_content='y 50% within the next decade. \nThe '),
    Document(metadata={}, page_content='ancient Egyptian pharaohs were know'),
    Document(metadata={}, page_content='n for their elaborate headdresses a'),
    Document(metadata={}, page_content='nd ornate jewelry.')
]

# Recursive Text Splitting

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 35, chunk_overlap=0) 
print(text_splitter.create_documents([text]))


[
    Document(metadata={}, page_content='The astronauts on the'),
    Document(metadata={}, page_content='International Space Station'),
    Document(metadata={}, page_content='conducted a spacewalk to repair a'),
    Document(metadata={}, page_content='malfunctioning solar panel.'),
    Document(metadata={}, page_content='The aroma of freshly baked'),
    Document(metadata={}, page_content='croissants wafted through the'),
    Document(metadata={}, page_content='charming French bakery.'),
    Document(metadata={}, page_content='The new policy aimed to reduce'),
    Document(metadata={}, page_content='carbon emissions by 50% within the'),
    Document(metadata={}, page_content='next decade.'),
    Document(metadata={}, page_content='The ancient Egyptian pharaohs were'),
    Document(metadata={}, page_content='known for their elaborate'),
    Document(metadata={}, page_content='headdresses and ornate jewelry.')
]

# Semantic Chunking

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings.ollama import OllamaEmbeddings

# Percentile - all differences between sentences are calculated, and then any difference greater than the 
# X percentile is split

#embed_function = OllamaEmbeddings(model="nomic-embed-text")
embed_function = OpenAIEmbeddings(model="nomic-embed-text")
text_splitter = SemanticChunker(
    embeddings=embed_function, 
    breakpoint_threshold_type="percentile", # "percentile" "standard_deviation", "interquartile",
    breakpoint_threshold_amount=60
)
documents = text_splitter.create_documents([text])
print(documents)


[
    Document(
        metadata={},
        page_content='\nThe astronauts on the International Space Station conducted a spacewalk to repair a 
malfunctioning solar panel. The aroma of freshly baked croissants wafted through the charming French bakery.'
    ),
    Document(
        metadata={},
        page_content='The new policy aimed to reduce carbon emissions by 50% within the next decade. The ancient 
Egyptian pharaohs were known for their elaborate headdresses and ornate jewelry.'
    )
]

# AI Split

In [8]:
from langchain_community.chat_models.ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


prompt_template="""Based on the following context anwer the question:
=============================
Context: {context}
==============================
Question: {question}

Extra instructions: 
1. DO NOT ALTER ANY SENTENCE IN THE CONTEXT. 
2. CONSIDER ALL THE SENTENCES IN THE CONTEXT. 
3. DO NOT INVENT ANY NEW CONTEXT
4. DO NOT SUMMARIZE THE CONTEXT WHEN PROVIDING A REPLY
"""
prompt:ChatPromptTemplate = ChatPromptTemplate.from_template(prompt_template)

#Use your local LLM 
#local_llm = ChatOllama(
#    verbose=True,
#    model="phi3", 
#    base_url="http://127.0.0.1:11434", 
#    temperature=0)
#or use a paid one much faster
local_llm = ChatOpenAI(temperature=0)

chain = (
        {"context": RunnablePassthrough(), "question": RunnablePassthrough()}
        | prompt
        | local_llm
        | StrOutputParser()
    )

result = chain.invoke({
    "question":"""
        Split the page_content based on semantic differences if you find necessary
        return your reply in the same schema of the context received""",
    "context": documents
    })
print(result)

#Expected execution time: 51s

The page_content can be split into two sections based on semantic differences:

Section 1:
- The astronauts on the International Space Station conducted a spacewalk to repair a malfunctioning solar panel.

Section 2:
- The aroma of freshly baked croissants wafted through the charming French bakery.
- The new policy aimed to reduce carbon emissions by 50% within the next decade.
- The ancient Egyptian pharaohs were known for their elaborate headdresses and ornate jewelry.